# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#Need to mount google drive for this to work
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
!git clone https://github.com/philferriere/cocoapi.git
!pip install mrcnn
!pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
!git clone https://github.com/matterport/Mask_RCNN



# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append('/content/Mask_RCNN/samples/coco/')
import coco
# import coco




%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Cloning into 'cocoapi'...
remote: Enumerating objects: 994, done.
remote: Total 994 (delta 0), reused 0 (delta 0), pack-reused 994
Receiving objects: 100% (994/994), 12.26 MiB | 6.02 MiB/s, done.
Resolving deltas: 100% (589/589), done.
     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for mrcnn: filename=mrcnn-0.2-cp36-none-any.whl size=54931 sha256=5ebc678b827144cb5fd6cb1e6fa73e04a1bfa55227a0fb03115f758eedce00f3
  Stored in directory: /root/.cache/pip/wheels/11/ed/28/e550ddc897c04c336b923eae4eb35c9aae993d20ce39d9cc40
Successfully built mrcnn
Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 116.75 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (567/567), done.


Using TensorFlow backend.


... done downloading pretrained model!


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [4]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

## Create Model and Load Trained Weights

In [5]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

AttributeError: ignored

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [0]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [0]:
#Given an image blur or sharpen everything except mask produced by the Faster_RCNN
#provide number of iterations, filter type 0 for blur 1 for sharpen, and the image

def mask_filter(it,ft,img):
  import cv2
  from google.colab.patches import cv2_imshow
#filename = '/content/Mask_RCNN/images/000000024144.jpg'


#image = skimage.io.imread(filename)
#print(image.shape)
# Run detection
  image=img        #skimage.io.imread('/content/drive/My Drive/Colab Notebooks/usable/000000024144.jpg')
  results = model.detect([image], verbose=1)
 # print(len(results))
  r=results[0]
# Visualize results

#a,b,c=r['masks'].shape true where mask is
#run model.detect
#apply a filter to the image ignoring where the mask is
#feed resulting image into model.detect
#use an array to keep track of scores
#array_scores=[]
#images_dir="/content/drive/My Drive/Colab Notebooks/usable/"
  
  #0 for blur 1 for sharpen
  if ft == 0: 
    kernel = np.ones((5,5),np.float32)/25
  elif ft == 1:
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])




#for imgs in os.listdir(images_dir):
  
 # images_dir=images_dir+imgs
 # scores=[]
#image=skimage.io.imread('/content/drive/My Drive/Colab Notebooks/usable/000000024144.jpg')
  temp = image


  for iteration in range(0,it):
    temp=cv2.filter2D(temp,-1,kernel)

#  print(temp.shape)
  
  y,x,z=r['masks'].shape
 # print(y,x,z)
  for j in range(y):
    for k in range(x):
      if r['masks'][j,k,0] == True:
        temp[j,k,0] = image[j,k,0]
        temp[j,k,1] = image[j,k,1]
        temp[j,k,2] = image[j,k,2]

  #temp=cv2.cvtColor(temp,cv2.COLOR_BGR2RGB)
  #return is the filtered image
  return temp
    
   # results = model.detect([temp], verbose=1)
   # scores.append(r['scores'])
   # print(iterations)
  #array_scores.append(scores)
 # cv2_imshow(temp)
  #images_dir="/content/drive/My Drive/Colab Notebooks/usable/"
#print(r['scores'])

#print(array_scores)




#r = results[0]
#visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
 #                         class_names, r['scores'])




In [0]:
from google.colab.patches import cv2_imshow
import cv2
import shutil
counter=0
images_dir="/content/drive/My Drive/Colab Notebooks/images2/"
for imgs in os.listdir(images_dir):  
  dif=[]
  
  images_dir=images_dir+imgs
  image=skimage.io.imread(images_dir)
  temp=image
  results = model.detect([temp], verbose=1)
  r = results[0]
  #visualize.display_instances(temp, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
  
  temp1=mask_filter(1000,1,temp)
  results = model.detect([temp1], verbose=1)
  r1=results[0]
  if len(r1["class_ids"]) != len(r["class_ids"]):
    counter=counter+1
  else:
    if r1["class_ids"] == r["class_ids"]:
      print("r1-r is: ",r1["scores"]-r["scores"])
    else:
      print("num objects same but class id not equal")
      visualize.display_instances(temp, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
      visualize.display_instances(temp1, r1['rois'], r1['masks'], r1['class_ids'], class_names, r1['scores'])
  

  images_dir="/content/drive/My Drive/Colab Notebooks/images2/"
#number of instances where there were extra objects detected
print("number of images with extra detections", counter)


In [0]:
""